# Readme file explaining how to set up k223d

## Description of k223d

_k223d_ produces fractal stochastic slip distributions on non-planar faults that are described by triangular mesh. The programme will also provide the rupture time for each location on the fault plane for a given nucleation location and rupture velocity. It is possible to assign surface nodes 

_k223d_ is based on the composite source model technique ([Zeng et al. BSSA, 1994](https://agupubs.onlinelibrary.wiley.com/doi/abs/10.1029/94GL00367)) whereby a set of hierarchical circular patches of slip are randomly placed on the fault plane based on a spatial probablity density function. Each individual patch of slip has a spatial distribution defined as a circular crack ([Ruiz et al., GJI, 2011](https://academic.oup.com/gji/article/186/1/226/697919)). At each point on the fault plane the contribution from the patches of slip at that location are summed together leading to the production of a final slip distribution that is self-similar. A further development has been the inclusion of surface rupture in _k223d_ when surface nodes are specified. In this newest version it is now possible to calculate the travel time of the rupture front across the fault plane using a user defined rupture velocity and a nucleation location.  

In summary this progamme will:
- provide a stochastic slip distribution for each cell. 
- generate surface rupture if requested
- calculate the rupture time across the fault for a given nucleation location and rupture velocity. 

This programme is based on the [slipk2](https://github.com/andherit/slipk2) code for the generation of fractal slip while the kernal for calculating the distance across an unstructured mesh is based on [trilateration](https://github.com/andherit/trilateration)

## Inputs
There are two main input files required to run the programme a _input_file_  and a file containing a triangular mesh in a vtk format. A detailed description of both input files is given below. To aid the user, the jupyter notebook _planar_mesh.ipynb_ in the Notebook folder provides an example of how to generate a mesh using [Gmsh](https://gmsh.info), set up the inputfile and run _k223d_. A description of how to setup a docker containing _k223d_ and the jupyter notebook is provided [below](##Setting-up-Docker-Image).

#### Setting the rupture size
There are two option on how to run the code, it can be assumed that the earthquake will occupy the whole mesh or a section of it based on a prescribed magnitude. The other option is that the area of the earthquake is defined by an empirical scalling relationship. At the moment the only option is [Strasser et al.,SRL, 2010](https://pubs.geoscienceworld.org/ssa/srl/article/81/6/941/143755/Scaling-of-the-Source-Dimensions-of-Interface-and) but others will be added with time. When this option is activated the cells around the designated nucleation location are choosen with more adjoining cells added until the rupture area is large than the value specified by the scaling relationship.


#### Surface rupture
Slip will automatically taper to zero at the edge of the mesh. In the case of edge representing the surface this is not always desirable. If nodes are defined as representing the earth's surface the slip will not be taper but will remain high with no decrease in slip at the surface. To know more about the technique used to achieve it see [Murphy and Herrero, GJI, 2020](https://academic.oup.com/gji/article/213/3/2060/4939268)

### Input file
_k223d_ automatically looks for a file named _"input_file"_. Many options can be left as they are in this file. For a full description of all the parameters are provided in the inputfile itself as comments. The more common parameters that the user will probably want to change are:
- **mesh_file** : complete name of the file containing the input mesh
- **magnitude** : size of the earthquake the user wishes to generate.
- **define_area** : this decides whether to use an emperical scaling relationship. Set equal to `true` for _k223d_ to automatically set the area of the fault based on value given for the `magnitude` whereby a Strasser et al. scaling relationship will be used to estimate the rupture area and the final slip distribution will be scaled to magnitude provided. If set to `false` _k223d_ will assume the whole mesh is being used for the slip distribution and the slip scaled to correspond to the magnitude specified.  
- **out_file** : specify the name of the output vtk file that will be generated by _k223d_. ".vtk" will be automatically added to the name provided


### Mesh file
_k223d_ searches for the mesh file specfied in the inputfile. _k223d_ reads in a [Legacy VTK Format](https://docs.vtk.org/en/latest/design_documents/VTKFileFormats.html), an example of how to construct a file using this format is provided in the Notebook with the corresponding python function provided as [well](notebook/notebook_utils.py#L259).  

Inside the mesh file, the order of the information in the vtk is as follows:
1) **Nodes** - an x,y,z list of the nodal coordinates
2) **Cells** - a 3 integer list connecting nodal indices (based on above list) to each element
3) **Velocity** - stored on the cells, this must be defined if time is provided and none of the values can be 0, this is an optional parameter.  
4) **Time** - stored on the nodes, set to -200 where rupture time is to be calculated and with an initial value on one node or a group of nodes where the earthquake initiates. 
5) **Surface** - list of integers that provides a list of indices related to nodes that are on the surface, this is an optional parameter, if it is not assigned it is assumed that mesh does not reach the surface.  


## Output 
The program outputs a vtk file. This file contains the mesh with the slip distribution and rupture velocity assigned on each cell while the rupture time is saved on the cell nodes. 

## Version History 
**Version 2:** Inclusion of rupture time. Standardise input and output of the mesh to a vtk file format. 

**Version 1:** Provide static slip distribution. 


## References 
If using _k223d_ we would appreciate if you could reference the following article:
- Herrero, A. and Murphy, S., 2018. Self-similar slip distributions on irregular shaped faults. Geophysical Journal International, 213(3), pp.2060-2070.

In the case of surface rupture the following reference is relavent:
- Murphy, S. and Herrero, A., 2020. Surface rupture in stochastic slip models. Geophysical Journal International, 221(2), pp.1081-1089.

Should you end up using the notebooks in the production of publications, the following references may need to be considered: 
- _Gmsh_ has been used in the generation of meshes, check their website on how best to reference them(https://gmsh.info). For exampel the authors appreciate referencing the following paper: C. Geuzaine and J.-F. Remacle, Gmsh: a three-dimensional finite element mesh generator with built-in pre- and post-processing facilities. International Journal for Numerical Methods in Engineering, Volume 79, Issue 11, pages 1309-1331, 2009.
- Graphics are produced using [Plotly](https://plotly.com/chart-studio-help/citations/)
- A number of other python libraries were used in certain notebooks such as [Meshio](https://zenodo.org/records/1288334), [GeoPandas](https://zenodo.org/records/3946761#.Xy24LC2ZPOQ), [PyProj4](https://zenodo.org/records/4571637).
- More fault meshes from the European fault-source model 2020 can be found here:  https://seismofaults.eu



## Using a Docker Container
There are two approaches that can be used to run _k223d_ through docker. Both methods will require the installation of [docker](https://www.docker.com)
The first and easier is to pull a container from Docker Hub at the following [repository](https://hub.docker.com/repository/docker/shanemurphy190/k223d/). The second is to build the docker image on your computer. To do this follow the instructions below. 


### Download _k223d_
Download clone _k223d_ which can be done using the following command: 
```
git clone https://github.com/s-murfy/k223d.git
```

### Build Docker Image
This can be done two ways, the first through a terminal in Docker or in a terminal of the operating system. In both cases the Docker software needs to be running. 
1. navigate to the k223d folder with the dockerfile. 
2. Type the following into the prompt:
    ```
    DOCKER_BUILDKIT=1 docker build -t k223d-image .
    ```
    if this gives an error, it is possible the BUILDKIT is already setup in which case the following will work:
    ```
    docker build -t k223d-image .
    ```
    Another source of error can be caused by having a VPN active when attempting to build the image. 

3. Compiling of the image can take a few minutes     

### Start container

With the container built we can now start the image. 

To do so, either press play in docker, after which a window will appear for optional settings, click down and add `8888` for the `host port` option.

<img src="setup_images/docker_option_settings.png" alt="example of how to set the external port" width="300"/>

- In terminal the following command achieves the same results :

```
docker run -p 8888:8888 -v $(pwd):/app k223d-image
```

In both cases lines of text are streamed to the screen either in docker or in the terminal. In this text look for the two lines with `http://` 

<img src="setup_images/ip_location.png" alt="address for notebook" width="500"/>

Copy one of them into the internet browers to start the jupyter notebook. Personally the `http://127.0.0.0.1` works best for me. 


# Running Notebooks
This involves two steps 
1. Compile _k223d_
2. Start a jupyter notebook

### Compile k223d
_k223d_ is written in fortran and there needs to be compiled with a fortran compiler. In the case of the docker example we will use gfortran whose libraries are preloaded in the docker environement. To compile the k223d, click on the terminal images in the notebook. And type the following commands:
```
cd codes 
make 
```
The _k223d_ should now compile without generating errors. 

### Open notebook 
After compiling _k223d_ compiled the Jupyter notebooks can be opened. These are found in `notebook` folder. There are two examples provided: in the folder `planar_eg` the file _planar_mesh.ipynb_ is a workflow on how to produce a planar mesh, run _k223d_ and view the results. In the folder `EFSM_eg` the file _read_EFSM20.ipynb_ provides an example on how to read a mesh from the EFSM database, remesh it, run _k223d_, view the results and generate an file that can be viewed in QGIS. 

## Exporting/Importing files from a container
If you want to copy a file created by _k223d_ or by one of the Notebooks, right click on the file in the notebook and select download and a copy of the file will be put in your Downloads folder. 

To import files inside the container, start the container and use the _docker cp_ command in a terminal to copy the file in. Below is an example of how to copy the file `test.txt` into the `/notebook/EFSM_eg` folder within in container:
```
docker cp test.txt ContainerID:app/notebook/EFSM_eg
```
